In [ ]:
%pip install -qU azure-ai-documentintelligence pandas

In [ ]:
# text splitting and chunking
import os
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from dotenv import load_dotenv

load_dotenv()
di_endpoint = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT")
di_credential = AzureKeyCredential(os.getenv("AZURE_DOCUMENT_INTELLIGENCE_KEY"))
document_intelligence_client = DocumentIntelligenceClient(di_endpoint, di_credential)

In [3]:
path_to_sample_documents = "data/azure-well-architected.pdf"

if not os.path.exists(path_to_sample_documents):
    raise FileNotFoundError(f"Sample document not found at {path_to_sample_documents}")

# read document and analyze layout
with open(path_to_sample_documents, "rb") as f:
    poller = document_intelligence_client.begin_analyze_document("prebuilt-layout", body = f)

result = poller.result()

In [22]:
import pandas as pd
import uuid

# Extract Text and Headings
pages = []

# Structure the documents as Pages -> Paragraphs -> Text
for paragraph in result.paragraphs:
    page = {}
    role = paragraph["role"] if "role" in paragraph else "paragraph"
    content = paragraph["content"] if "content" in paragraph else ""
    # if a title with "Architecture best practices for" is found, create a new page
    if "Architecture best practices for" in content and role == "title":
        # Extract headings and text
        page["id"] = str(uuid.uuid4())
        page["heading"] = content
        page["content"] = content
        pages.append(page)
    else:
        page = pages[-1]
        page["content"] += f"\n{content}"
print(f"Extracted {len(pages)} documents with headings and text.")

Extracted 22 documents with headings and text.


In [1]:
pd.DataFrame(pages).to_csv("data/azure-well-architected-pages.csv", index=False)

NameError: name 'pd' is not defined